# Battle of the Neigborhoods
## Capstone Project - Applied Data Science specialization

Author - Abhinav Paul || Created on 27/04/2020

### Problem Statement


Exploring Bangalore for most busiest part of the District based on Checkins from Foursquare 

### Data Description

Neighborhood Data -  https://data.gov.in/catalog/all-india-pincode-directory?filters%5Bfield_catalog_reference%5D=85840&format=json&offset=0&limit=6&sort%5Bcreated%5D=desc



In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

import os #to change directory

import folium # plotting library

print('Libraries imported.')

Libraries imported.


Data fetch from net and Preparation from https://data.gov.in/catalog/all-india-pincode-directory?filters%5Bfield_catalog_reference%5D=85840&format=json&offset=0&limit=6&sort%5Bcreated%5D=desc

In [193]:
os.chdir('/Users/abhinavpaul/Documents/GitHub/Coursera_capstone/Capstone project')
#os.getcwd() # to check the change
# importing and filtering for Bangalore District Data into a dataframe
path = '{}/all_india_PO_list_without_APS_offices_ver2_lat_long.csv'.format(os.getcwd())
raw_data = pd.read_csv(path).dropna(subset=['longitude','latitude']).reset_index()
Bang_raw_data = raw_data[(raw_data['Districtname']=='Bangalore')].reset_index()
Bangalore_data = Bang_raw_data[['pincode','officename','Districtname','longitude','latitude']].dropna()
Bangalore_data['officename']=Bangalore_data['officename'].replace('.+ S.O','.+')
Bangalore_data.rename(columns={'pincode':'PostalCode','officename':'Neighborhood','Districtname':'City'},inplace=True)
Bangalore_data.head()



,PostalCode,Neighborhood,City,longitude,latitude
0,560045,Arabic College S.O,Bangalore,77.6206,13.0291
1,560103,Bellandur S.O,Bangalore,77.6760,12.9298
2,560071,Domlur S.O,Bangalore,77.6359,12.9611
3,560077,Dr. Shivarama Karanth Nagar S.O,Bangalore,77.6293,13.0681
4,560005,Fraser Town S.O,Bangalore,77.6164,13.0005


In [174]:
address = 'Bangalore'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))



# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Bangalore_data['latitude'], Bangalore_data['longitude'], Bangalore_data['Neighborhood'],Bangalore_data['City']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

/Users/abhinavpaul/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


Foursquare credentials

In [129]:
CLIENT_ID = 'TBPLKSLPDCOMMX2YIDPJ5UG2NRWHG4YOB3VXMPJI5YYP54H3' # your Foursquare ID
CLIENT_SECRET = 'OV15HD3ZH2152BWBWDWFSKBEONQJNSQCJAI0S4BO0WLFWLPE' # your Foursquare Secret
VERSION = '20200101' # Foursquare API version

print('Credentails Uploaded')

Credentails Uploaded


In [ ]:
def address_call (address) :
    geolocator = Nominatim()
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The coordinates of {} are : {}, {}'.format(address,latitude, longitude))
    return latitude, longitude

In [ ]:
city = ['Delhi','Kolkata','Mumbai','Bangalore']
a =[]
for j in city:
    a = pd.DataFrame(address_call(j))

In [175]:

ACCESS_TOKEN = 'WC4JYNHNPRJSZU2QOPWKCPDVOFGDIU5S2C4CPIX4GRDKZDGP'

def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=150):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues, results

# type your answer here
Bangalore_venues ,Bang_result = getNearbyVenues(names=Bangalore_data['Neighborhood'],
                                   latitudes=Bangalore_data['latitude'],
                                   longitudes=Bangalore_data['longitude']
                                  )

Arabic College S.O
Bellandur S.O
Domlur S.O
Dr. Shivarama Karanth Nagar S.O
Fraser Town S.O
G.K.V.K. S.O
H.A.L II Stage H.O
HighCourt S.O
Jeevanbhimanagar S.O
Kothanur S.O
Mahatma Gandhi Road S.O
Marathahalli Colony S.O
NAL S.O
New Thippasandra S.O
Sadashivanagar S.O
Sahakaranagar P.O S.O
Vimanapura S.O
Yelahanka S.O
Yelahanka Satellite Town S.O
Bangalore G.P.O. 
Ashoknagar S.O (Bangalore)
B Sk II Stage S.O
Bannerghatta Road S.O
Basavanagudi H.O
Bommanahalli S.O (Bangalore)
Bommasandra Industrial Estate S.O
Carmelram S.O
Chandapura S.O
Dharmaram College S.O
Electronics City S.O
Gottigere S.O
HSR Layout S.O
Hulimavu S.O
J P Nagar S.O
Jayanagar H.O
Jayangar III Block S.O
JP Nagar III Phase S.O
Koramangala VI Bk S.O
Mico Layout S.O
Padmanabhnagar S.O
St. John's Medical College S.O
Gayathrinagar S.O
Jalahalli East S.O
Jalahalli H.O
Mahalakshmipuram Layout S.O
Malleswaram S.O
Malleswaram West S.O
Mathikere S.O
Msrit S.O
Palace Guttahalli S.O
Rajajinagar H.O
Rajajinagar IVth Block S.O
Swimmi

In [170]:
Bangalore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bellandur S.O,12.9298,77.676,Kicks On Grass,12.930045,77.679679,Soccer Field
1,Bellandur S.O,12.9298,77.676,Cafe Coffee Day Central 3,12.926107,77.675755,Café
2,Bellandur S.O,12.9298,77.676,McDonald's,12.927228,77.675688,Fast Food Restaurant
3,Bellandur S.O,12.9298,77.676,Domino's Pizza,12.926045,77.676107,Pizza Place
4,Bellandur S.O,12.9298,77.676,KFC,12.925980,77.675349,Fast Food Restaurant


In [171]:
Bang_result

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4fe6b2bee4b0756d12fc44bf',
   'name': 'Lavonne',
   'location': {'address': 'No.151',
    'crossStreet': '2nd Cross, Domlur, 2nd Stage',
    'lat': 12.963908917512304,
    'lng': 77.63857909555004,
    'labeledLatLngs': [{'label': 'display',
      'lat': 12.963908917512304,
      'lng': 77.63857909555004}],
    'distance': 426,
    'postalCode': '560071',
    'cc': 'IN',
    'city': 'Bangalore',
    'state': 'Karnātaka',
    'country': 'India',
    'formattedAddress': ['No.151 (2nd Cross, Domlur, 2nd Stage)',
     'Bangalore 560071',
     'Karnātaka',
     'India']},
   'categories': [{'id': '4bf58dd8d48988d16d941735',
     'name': 'Café',
     'pluralName': 'Cafés',
     'shortName': 'Café',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'},
     'primary': True}],
   'p

In [173]:
# one hot encoding
manhattan_onehot = pd.get_dummies(Bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = Bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Bakery,Bookstore,Breakfast Spot,Café,Fast Food Restaurant,Food Court,Gym,Indian Restaurant,Kerala Restaurant,Lounge,Multiplex,Pizza Place,Shopping Mall,Soccer Field,Travel & Transport
0,Bellandur S.O,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Bellandur S.O,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Bellandur S.O,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Bellandur S.O,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Bellandur S.O,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
